# Test PKG

In [1]:
import sys
from pathlib import Path
# sys.path.insert(1, str(Path(".").absolute() / 'src')) # To subdir
sys.path.insert(1, '../src')
import peddesign
import numpy as np

## Design CT

### General Design

In [2]:
d_ctchest1 = peddesign.design_ct_str("chest", 10, is_first_study=False)
d_ctchest1

'-- Design CT --\nCT chest, venous only\nBody weight: 10 kg\nkV: 80\nmA: auto\nNoise index: 20\nDelay: 45 sec\nContrast: 10 ml (1 ml/kg * 10 kg = 10)\nRate: 0.9 ml/sec (10 + 15) / 30 = 0.833\n---\n'

In [3]:
type(d_ctchest1)

str

In [4]:
peddesign.design_ct("chest", 10, is_first_study=False)

-- Design CT --
CT chest, venous only
Body weight: 10 kg
kV: 80
mA: auto
Noise index: 20
Delay: 45 sec
Contrast: 10 ml (1 ml/kg * 10 kg = 10)
Rate: 0.9 ml/sec (10 + 15) / 30 = 0.833
---



### CT Chest

In [5]:
ctchest1 = peddesign.DesignCTchest(10, True)
print(ctchest1)

-- Design CT --
CT chest, venous only
Body weight: 10 kg
kV: 80
mA: auto
Noise index: 17
Delay: 45 sec
Contrast: 10 ml (1 ml/kg * 10 kg = 10)
Rate: 0.9 ml/sec (10 + 15) / 30 = 0.833
---



In [6]:
ctchest1.str_design

'-- Design CT --\nCT chest, venous only\nBody weight: 10 kg\nkV: 80\nmA: auto\nNoise index: 17\nDelay: 45 sec\nContrast: 10 ml (1 ml/kg * 10 kg = 10)\nRate: 0.9 ml/sec (10 + 15) / 30 = 0.833\n---\n'

### CTWA

In [6]:
peddesign.design_whole_abd_2(19, False, rate_formula= "no_delay")

-- Design CT -- 

CT whole abdomen, venous only
Body weight: 19 kg
kV: 80
mA: auto
Noise index: 17
Delay: 60 or 65 or 70 sec
Contrast: 38 ml (2 ml/kg * 19 kg = 38) 
Rate: 1.2 ml/sec (38 + 15) / 45 = 1.178

---


In [7]:
peddesign.design_whole_abd_2(20, False, rate_formula= "delay", delay_sec=65)

-- Design CT -- 

CT whole abdomen, venous only
Body weight: 20 kg
kV: 80 or 100
mA: auto
Noise index: 17
Delay: 65 sec
Contrast: 40 ml (2 ml/kg * 20 kg = 40) 
Rate: 1.3 ml/sec (40 + 15) / (65 - 20) = 1.222

---


In [8]:
ctwa1 = peddesign.DesignCTwholeAbd(20, False, rate_formula= "delay", delay_sec=65)
ctwa1


-- Design CT --
CT whole abdomen, venous only
Body weight: 20 kg
kV: 80 or 100
mA: auto
Noise index: 17
Delay: Delay: 65 sec 
Contrast: 40 ml (2 ml/kg * 20 kg = 40)
Rate: 1.3 ml/sec (40 + 15) / (65 - 20) = 1.222
---

In [9]:
ctwa1.str_design

'\n-- Design CT --\nCT whole abdomen, venous only\nBody weight: 20 kg\nkV: 80 or 100\nmA: auto\nNoise index: 17\nDelay: Delay: 65 sec \nContrast: 40 ml (2 ml/kg * 20 kg = 40)\nRate: 1.3 ml/sec (40 + 15) / (65 - 20) = 1.222\n---\n'

### CT chest + WA

In [10]:
peddesign.DesignCTchestWholeAbd(20, False, rate_formula= "delay", delay_sec=65)


-- Design CT --
Venous chest + whole abdomen
Body weight: 20 kg
kV: 80 or 100
mA: auto
Noise index: 17
Delay: Delay: 65 sec 
Contrast: 40 ml (2 ml/kg * 20 kg = 40)
Rate: 1.3 ml/sec (40 + 15) / (65 - 20) = 1.222
---

### CTA Liver

In [11]:
peddesign.DesignCTAliver(10)


-- Design CT --
CTA liver
Body weight: 10 kg
kV: 80
mA: auto
Noise index: 17
Delay: 20 sec (CTA); 70 sec (Venous)
Contrast: 25.0 ml (2.5 ml/kg * 10 kg = 25.0)
Rate: 2.0 ml/sec (25.0 + 15) / 20 = 2.0
---

## Child

### Get kV

In [13]:

def get_kV(weight_kg: float) -> str:
    """Get kV according to weight in kg

    Args:
        weight_kg (float): patient's weight in kg

    Returns:
        str: string of weight 
    """

    assert isinstance(weight_kg, (int, float)) and weight_kg > 0
    
    if weight_kg < 20:
        kV = "80"
    elif weight_kg == 20:
        kV = "80 or 100"
    elif weight_kg < 45:
        kV = "100"
    elif weight_kg == 45:
        kV = "100 or 120"
    elif weight_kg <= 60:
        kV = "120"
    else:
        # weight_kg ≥ 60
        kV = "120 ?"
    
    return kV


In [14]:
help(get_kV)

Help on function get_kV in module __main__:

get_kV(weight_kg: float) -> str
    Get kV according to weight in kg
    
    Args:
        weight_kg (float): patient's weight in kg
    
    Returns:
        str: string of weight



In [18]:
get_kV(True)

'80'

In [20]:
isinstance(True, int)

True

### Get Contrast ML

In [3]:

def get_contrast_ml(ml_kg: float, weight_kg: float) -> float:
    """Get contrast volume in ml from ml/kg and weight (kg)

    Args:
        ml_kg (float): Contrast ml/kg 
        weight_kg (float): Patient's weight (kg)

    Returns:
        float: ml of contrast
    """
    assert isinstance(weight_kg, (int, float)) and weight_kg > 0
    
    ml = ml_kg * weight_kg
    
    # Maximum contrast = 80 ml
    return round(ml, 1) if ml < 80 else 80



### Print Contrast

In [4]:
def print_contrast(ml_kg: float, weight_kg: float) -> None:
    """Print contrast calculation to STD out

    Args:
        ml_kg (float): Contrast ml/kg 
        weight_kg (float): Patient's weight (kg)
    """
    ml = ml_kg * weight_kg
    if ml >= 80:
        ml_adj_txt = "80 ml [maximum]"
    else:
        ml_adj_txt = f"{round(ml, 1)} ml"
    
    print(f"Contrast: {ml_adj_txt} ({ml_kg} ml/kg * {weight_kg} kg = {ml}) ")



### Rate (Ml/sec)

In [ ]:
def print_rate_ml_sec_2(contrast_ml, rate_formula="no_delay", delay_sec=None):
    rate_formula = rate_formula.lower()
    assert isinstance(contrast_ml, (int, float)) and contrast_ml > 0
    
    if rate_formula == "no_delay":
        rate = (contrast_ml + 15) / 45
        rate_adj = math.ceil(rate * 10) / 10  # Round up to 1 decimal place
        
        show_calc = f"({contrast_ml} + 15) / 45 = {round(rate, 3)}"
    
    elif rate_formula == "delay":
        assert isinstance(delay_sec, (int, float))
        
        rate = (contrast_ml + 15) / (delay_sec - 15)
        rate_adj = math.ceil(rate * 10) / 10  # Round up to 1 decimal place
        
        show_calc = f"({contrast_ml} + 15) / ({delay_sec} - 15) = {round(rate, 3)}"
        
    else:
        raise ValueError("Invalid rate_formula value. Please choose 'no_delay' or 'delay'.")
    
    print(f"Rate: {rate_adj} ml/sec {show_calc}")

In [48]:
# Your arrays
array1 = np.array([60, 65, 70])
array2 = np.array([20, 25])

array1.reshape(-1, 1) - array2

array([[40, 35],
       [45, 40],
       [50, 45]])

## Ideas

In [24]:
def print_something(x, y, z):
    print(f"""
Hello 
    x = {x}
    y = {y}
    z = {z}
    """)

In [25]:
print_something(1, 2, 3)


Hello 
    x = 1
    y = 2
    z = 3
    


In [26]:
strings1 = """
Hello 
    x = {x}
    y = {y}
    z = {z}
"""

def print_something2(x, y, z):
    # Use str.format() to replace placeholders with argument values
    formatted_string = strings1.format(x=x, y=y, z=z)
    print(formatted_string)

In [27]:
print_something2(1,2,3)


Hello 
    x = 1
    y = 2
    z = 3



In [5]:
some_str = {
    "string1": """
Hello 
    x = {x}
    y = {y}
    z = {z}
""",
    "string2": """
Hi {x}, {y}, {z}
"""
}


def print_something3(x, y, z):
    # Use str.format() to replace placeholders with argument values
    fmt_str1 = some_str["string1"].format(x=x, y=y, z=z)
    fmt_str2 = some_str["string2"].format(x=x, y=y, z=z)
    print(fmt_str1, fmt_str2)

In [6]:
print_something3(1, 2, 3)


Hello 
    x = 1
    y = 2
    z = 3
 
Hi 1, 2, 3



In [9]:
def get_contrast_str(ml_kg: float, weight_kg: float) -> None:
    """Get contrast calculation as string

    Args:
        ml_kg (float): Contrast ml/kg 
        weight_kg (float): Patient's weight (kg)
    """
    ml = ml_kg * weight_kg
    if ml >= 80:
        ml_adj_txt = "80 ml [maximum]"
    else:
        ml_adj_txt = f"{round(ml, 1)} ml"
    
    contrast_calc = f"{ml_adj_txt} ({ml_kg} ml/kg * {weight_kg} kg = {ml})"
    return contrast_calc

In [10]:
get_contrast_str(1, 2)

'2 ml (1 ml/kg * 2 kg = 2)'